In [70]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle
import os
import pathlib
from pathlib import Path

In [ ]:
import datetime

In [ ]:
datetime.datetime.now().strftime('%Y%m%d-%H%M')

In [ ]:
le_gender = LabelEncoder()
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop='first')
scaler = StandardScaler(with_mean=False)

In [ ]:
pd.read_csv()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [73]:
data_path = os.path.join(Path.cwd().parent.as_posix(),'data','Churn_Modelling.csv')

data = pd.read_csv(data_path)[['CreditScore', 'Geography','Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard','IsActiveMember', 'EstimatedSalary', 'Exited']]
display(data.info())
data['Gender'] = le_gender.fit_transform(data['Gender'])  
X_geo = ohe.fit_transform(data[['Geography']])
geo_df = pd.DataFrame(
    X_geo,
    columns=ohe.get_feature_names_out(['Geography']),
    index=data.index
)
data[['Geography_A','Geography_B']] = geo_df.astype(int)
data = data.drop(columns='Geography')
X = data.drop(columns='Exited')
y = data[['Exited']]
del X_geo, geo_df, data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)

svm = SVC(kernel="rbf", class_weight="balanced")  # quita class_weight si no hay desbalance
svm.fit(X_train,y_train)

accuracy = svm.score(X_train,y_train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


None

/home/sda/env_glob/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
os.makedirs('../pkl',exist_ok=True)

with open("../pkl/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

with open("../pkl/ohe.pkl", "wb") as f:
    pickle.dump(ohe, f)

with open("../pkl/le_gender.pkl", "wb") as f:
    pickle.dump(le_gender, f)

# valid

In [74]:
import os

In [75]:
from pathlib import Path
daat = pd.read_csv(os.path.join(Path.cwd().parent.as_posix(),'data','validation.csv'))
daat

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_A,Geography_B,Exited
0,6.128411,0.000000,3.636791,0.692245,2.280609,1.703106,0.000000,2.00109,2.084866,0.000000,0.00000,NaN
1,6.916498,2.010625,4.211021,2.768979,1.959867,1.703106,2.201188,2.00109,1.599924,0.000000,0.00000,NaN
2,7.227585,2.010625,2.775446,0.000000,2.767928,1.703106,0.000000,0.00000,2.471036,2.319277,0.00000,NaN
3,8.388975,2.010625,3.923906,0.346122,1.277530,3.406212,2.201188,0.00000,2.887107,2.319277,0.00000,0.0
4,8.814127,0.000000,2.775446,0.346122,0.000000,3.406212,2.201188,2.00109,3.450355,0.000000,2.30481,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,7.652737,2.010625,3.349676,1.038367,0.000000,1.703106,2.201188,2.00109,0.272735,0.000000,2.30481,NaN
1996,7.725323,0.000000,3.445381,3.115102,0.000000,1.703106,2.201188,0.00000,0.341646,0.000000,0.00000,NaN
1997,7.963823,2.010625,4.689547,0.346122,2.137886,1.703106,2.201188,0.00000,1.784408,2.319277,0.00000,0.0
1998,6.688367,2.010625,4.689547,0.692245,0.000000,3.406212,0.000000,0.00000,0.174667,0.000000,2.30481,NaN


In [76]:
os.path.join(Path.cwd().parent.as_posix(),'pkl','model.pkl')

'/home/sda/Dropbox/Estudio!/Maestria/MLOps/mod3/taller4/Taller4-MLflow-Github-Actions/pkl/model.pkl'

In [77]:
with open(os.path.join(Path.cwd().parent.as_posix(),'pkl','model.pkl'), "rb") as f:
    model = pickle.load(f)
model

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,'balanced'
,verbose,False


In [78]:
model.predict(daat.drop(columns='Exited'))

array([0, 0, 0, ..., 1, 0, 1])

In [79]:
data = pd.read_csv(os.path.join(Path.cwd().parent.as_posix(),'data','validation.csv'))
X_valid = data.drop(columns='Exited')
y_valid = data[['Exited']]

In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      2000 non-null   float64
 1   Gender           2000 non-null   float64
 2   Age              2000 non-null   float64
 3   Tenure           2000 non-null   float64
 4   Balance          2000 non-null   float64
 5   NumOfProducts    2000 non-null   float64
 6   HasCrCard        2000 non-null   float64
 7   IsActiveMember   2000 non-null   float64
 8   EstimatedSalary  2000 non-null   float64
 9   Geography_A      2000 non-null   float64
 10  Geography_B      2000 non-null   float64
 11  Exited           414 non-null    float64
dtypes: float64(12)
memory usage: 187.6 KB


In [61]:
X_valid

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_A,Geography_B
0,0.065269,4.042611,0.238146,0.119801,0.000000,2.900569,4.845228,0.000000,0.000009,0.000000,0.000000
1,0.056345,4.042611,0.293103,0.119801,0.000037,2.900569,0.000000,0.000000,0.000057,0.000000,0.000000
2,0.078818,0.000000,0.348059,0.359402,0.000041,2.900569,0.000000,0.000000,0.000006,0.000000,0.000000
3,0.091399,0.000000,0.293103,0.599004,0.000027,2.900569,4.845228,0.000000,0.000037,0.000000,0.000000
4,0.091399,4.042611,0.311421,0.239601,0.000019,2.900569,4.845228,4.004361,0.000035,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
1995,0.064087,4.042611,0.613683,0.718804,0.000000,5.801139,4.845228,4.004361,0.000042,0.000000,5.312148
1996,0.063979,4.042611,0.329740,0.718804,0.000022,2.900569,4.845228,4.004361,0.000008,0.000000,5.312148
1997,0.086775,4.042611,0.393856,0.119801,0.000027,5.801139,4.845228,0.000000,0.000011,0.000000,0.000000
1998,0.077420,0.000000,0.338900,0.239601,0.000031,5.801139,4.845228,4.004361,0.000042,0.000000,0.000000


In [60]:
y_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Exited  414 non-null    float64
dtypes: float64(1)
memory usage: 15.8 KB


In [66]:
aaaa = pd.read_csv('../data/validation.csv').dropna()
aaaa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 414 entries, 3 to 1997
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      414 non-null    float64
 1   Gender           414 non-null    float64
 2   Age              414 non-null    float64
 3   Tenure           414 non-null    float64
 4   Balance          414 non-null    float64
 5   NumOfProducts    414 non-null    float64
 6   HasCrCard        414 non-null    float64
 7   IsActiveMember   414 non-null    float64
 8   EstimatedSalary  414 non-null    float64
 9   Geography_A      414 non-null    float64
 10  Geography_B      414 non-null    float64
 11  Exited           414 non-null    float64
dtypes: float64(12)
memory usage: 42.0 KB


# mlflow

In [9]:
import os
import pathlib
from pathlib import Path

In [ ]:
data = pd.read_csv(os.path.join(Path.cwd().parent.as_posix(),'data','validation.csv'))

In [10]:
Path.cwd().parent.as_posix()

'/home/sda/Dropbox/Estudio!/Maestria/MLOps/mod3/taller4/Taller4-MLflow-Github-Actions'

In [11]:
# workspace_dir = os.getcwd()  # Debería ser /home/runner/work/mlflow-deploy/mlflow-deploy
workspace_dir = Path.cwd().parent.as_posix()  # Debería ser /home/runner/work/mlflow-deploy/mlflow-deploy
mlruns_dir = os.path.join(workspace_dir, "mlruns")
tracking_uri = "file://" + os.path.abspath(mlruns_dir)
# Definir explícitamente la ubicación base deseada para los artefactos
artifact_location = "file://" + os.path.abspath(mlruns_dir)

In [15]:
artifact_location

'file:///home/sda/Dropbox/Estudio!/Maestria/MLOps/mod3/taller4/Taller4-MLflow-Github-Actions/mlruns'